# Enter State Farm

In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [28]:
%matplotlib inline
from __future__ import print_function, division
#path = "data/state-farm/"
path = "data/state-farm/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

In [3]:
batch_size=64

## Create sample

The following assumes you've already created your validation set - remember that the training and validation set should contain *different drivers*, as mentioned on the Kaggle competition page.

In [29]:
%cd data/state-farm

/home/ubuntu/deep/nbs/data/state-farm


In [31]:
%cd train

/home/ubuntu/deep/nbs/data/state-farm/train


In [6]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [7]:
for d in glob('c?'):
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)

In [8]:
from shutil import copyfile

In [9]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1500): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [13]:
import csv

In [27]:
with open(path+'driver_imgs_list.csv', 'rb') as csvfile:
    imgreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in imgreader:
        if(row[0] == 'p042' or row[0] == 'p002' or row[0] == 'p012'):
            os.rename(path+'train/'+row[1]+'/'+row[2], path + 'valid/'+row[1]+'/'+row[2])

In [11]:
%mkdir ../valid

In [32]:
%cd ../valid

/home/ubuntu/deep/nbs/data/state-farm/valid


In [33]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [34]:
%cd ../../..

/home/ubuntu/deep/nbs


In [36]:
%mkdir data/state-farm/results

In [37]:
%mkdir data/state-farm/sample/test

## Create batches

In [39]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [42]:
?get_classes()

In [43]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filename) = get_classes(path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


## Basic models

### Linear model

First, we try the simplest model and use default parameters. Note the trick of making the first layer a batchnorm layer - that way we don't have to worry about normalizing the input ourselves.

In [45]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])

As you can see below, this training is going nowhere...

In [46]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=6, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/6
1500/1500 [==============================] - 32s - loss: 11.6056 - acc: 0.2053 - val_loss: 12.8674 - val_acc: 0.1950
Epoch 2/6
1500/1500 [==============================] - 24s - loss: 12.0572 - acc: 0.2407 - val_loss: 11.8542 - val_acc: 0.2500
Epoch 3/6
1500/1500 [==============================] - 24s - loss: 11.9567 - acc: 0.2453 - val_loss: 12.2868 - val_acc: 0.2280
Epoch 4/6
1500/1500 [==============================] - 24s - loss: 11.6605 - acc: 0.2653 - val_loss: 11.8869 - val_acc: 0.2480
Epoch 5/6
1500/1500 [==============================] - 24s - loss: 12.1781 - acc: 0.2373 - val_loss: 12.0917 - val_acc: 0.2460
Epoch 6/6
1500/1500 [==============================] - 23s - loss: 11.7470 - acc: 0.2667 - val_loss: 12.7306 - val_acc: 0.1990


Let's first check the number of parameters to see that there's enough parameters to find some useful relationships:

In [47]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_2 (BatchNormal(None, 3, 224, 224)   6           batchnormalization_input_2[0][0] 
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 150528)        0           batchnormalization_2[0][0]       
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 10)            1505290     flatten_2[0][0]                  
Total params: 1505296
____________________________________________________________________________________________________


Over 1.5 million parameters - that should be enough. Incidentally, it's worth checking you understand why this is the number of parameters in this layer:

In [48]:
10*3*224*224

1505280

Since we have a simple model with no regularization and plenty of parameters, it seems most likely that our learning rate is too high. Perhaps it is jumping to a solution where it predicts one or two classes with high confidence, so that it can give a zero prediction to as many classes as possible - that's the best approach for a model that is no better than random, and there is likely to be where we would end up with a high learning rate. So let's check:

In [49]:
np.round(model.predict_generator(batches, batches.N)[:10],2)

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]], dtype=float32)

Our hypothesis was correct. It's nearly always predicting class 1 or 6, with very high confidence. So let's try a lower learning rate:

In [50]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=6, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/6
1500/1500 [==============================] - 31s - loss: 2.4690 - acc: 0.1693 - val_loss: 4.1586 - val_acc: 0.1520
Epoch 2/6
1500/1500 [==============================] - 24s - loss: 1.8092 - acc: 0.3967 - val_loss: 2.5331 - val_acc: 0.2870
Epoch 3/6
1500/1500 [==============================] - 24s - loss: 1.4544 - acc: 0.5480 - val_loss: 1.7188 - val_acc: 0.4010
Epoch 4/6
1500/1500 [==============================] - 24s - loss: 1.1710 - acc: 0.6813 - val_loss: 1.3605 - val_acc: 0.5880
Epoch 5/6
1500/1500 [==============================] - 25s - loss: 1.0070 - acc: 0.7567 - val_loss: 1.1968 - val_acc: 0.6310
Epoch 6/6
1500/1500 [==============================] - 24s - loss: 0.8480 - acc: 0.8193 - val_loss: 1.0457 - val_acc: 0.7050


Great - we found our way out of that hole... Now we can increase the learning rate and see where we can get to.

In [51]:
model.optimizer.lr=0.001

In [52]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 31s - loss: 0.7314 - acc: 0.8660 - val_loss: 0.9777 - val_acc: 0.7220
Epoch 2/4
1500/1500 [==============================] - 24s - loss: 0.6508 - acc: 0.8773 - val_loss: 0.8651 - val_acc: 0.7760
Epoch 3/4
1500/1500 [==============================] - 25s - loss: 0.5648 - acc: 0.9067 - val_loss: 0.8317 - val_acc: 0.7750
Epoch 4/4
1500/1500 [==============================] - 24s - loss: 0.5108 - acc: 0.9167 - val_loss: 0.7715 - val_acc: 0.8110


We're stabilizing at validation accuracy of 0.39. Not great, but a lot better than random. Before moving on, let's check that our validation set on the sample is large enough that it gives consistent results:

In [53]:
rnd_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=True)

Found 1000 images belonging to 10 classes.


In [54]:
val_res = [model.evaluate_generator(rnd_batches, rnd_batches.nb_sample) for i in range(10)]
np.round(val_res, 2)

array([[ 0.77,  0.81],
       [ 0.76,  0.82],
       [ 0.74,  0.82],
       [ 0.8 ,  0.8 ],
       [ 0.75,  0.82],
       [ 0.79,  0.81],
       [ 0.76,  0.82],
       [ 0.76,  0.82],
       [ 0.77,  0.81],
       [ 0.79,  0.8 ]])

Yup, pretty consistent - if we see improvements of 3% or more, it's probably not random, based on the above samples.

### L2 regularization

The previous model is over-fitting a lot, but we can't use dropout since we only have one layer. We can try to decrease overfitting in our model by adding [l2 regularization](http://www.kdnuggets.com/2015/04/preventing-overfitting-neural-networks.html/2) (i.e. add the sum of squares of the weights to our loss function):

In [55]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax', W_regularizer=l2(0.01))
    ])
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 31s - loss: 5.8829 - acc: 0.2727 - val_loss: 8.2246 - val_acc: 0.2930
Epoch 2/2
1500/1500 [==============================] - 23s - loss: 2.9203 - acc: 0.6087 - val_loss: 5.5870 - val_acc: 0.4290


In [56]:
model.optimizer.lr=0.001

In [57]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 31s - loss: 1.6070 - acc: 0.7580 - val_loss: 2.2116 - val_acc: 0.6910
Epoch 2/4
1500/1500 [==============================] - 25s - loss: 1.1089 - acc: 0.8433 - val_loss: 1.0764 - val_acc: 0.7590
Epoch 3/4
1500/1500 [==============================] - 25s - loss: 0.6457 - acc: 0.9093 - val_loss: 1.2196 - val_acc: 0.6890
Epoch 4/4
1500/1500 [==============================] - 24s - loss: 0.3756 - acc: 0.9573 - val_loss: 0.5784 - val_acc: 0.8520


Looks like we can get a bit over 50% accuracy this way. This will be a good benchmark for our future models - if we can't beat 50%, then we're not even beating a linear model trained on a sample, so we'll know that's not a good approach.

### Single hidden layer

The next simplest model is to add a single hidden layer.

In [58]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

model.optimizer.lr = 0.01
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/2
1500/1500 [==============================] - 31s - loss: 1.9901 - acc: 0.3527 - val_loss: 7.6179 - val_acc: 0.1540
Epoch 2/2
1500/1500 [==============================] - 24s - loss: 1.0338 - acc: 0.7207 - val_loss: 4.1208 - val_acc: 0.2390
Epoch 1/5
1500/1500 [==============================] - 31s - loss: 0.5756 - acc: 0.8787 - val_loss: 1.6300 - val_acc: 0.5080
Epoch 2/5
1500/1500 [==============================] - 25s - loss: 0.3564 - acc: 0.9473 - val_loss: 1.3849 - val_acc: 0.5830
Epoch 3/5
1500/1500 [==============================] - 24s - loss: 0.2455 - acc: 0.9740 - val_loss: 0.8627 - val_acc: 0.7430
Epoch 4/5
1500/1500 [==============================] - 24s - loss: 0.1627 - acc: 0.9913 - val_loss: 0.7316 - val_acc: 0.7830
Epoch 5/5
1500/1500 [==============================] - 24s - loss: 0.1117 - acc: 0.9960 - val_loss: 0.6122 - val_acc: 0.8510


Not looking very encouraging... which isn't surprising since we know that CNNs are a much better choice for computer vision problems. So we'll try one.

### Single conv layer

2 conv layers with max pooling followed by a simple dense network is a good simple CNN to start with:

In [75]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.01
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [72]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 33s - loss: 2.3575 - acc: 0.2333 - val_loss: 2.5227 - val_acc: 0.1070
Epoch 2/2
1500/1500 [==============================] - 28s - loss: 1.4265 - acc: 0.5713 - val_loss: 2.3518 - val_acc: 0.1110
Epoch 1/4
1500/1500 [==============================] - 36s - loss: 0.8833 - acc: 0.7913 - val_loss: 2.2567 - val_acc: 0.1390
Epoch 2/4
1500/1500 [==============================] - 27s - loss: 0.5933 - acc: 0.9067 - val_loss: 2.1721 - val_acc: 0.1870
Epoch 3/4
1500/1500 [==============================] - 26s - loss: 0.3998 - acc: 0.9573 - val_loss: 2.0925 - val_acc: 0.2660
Epoch 4/4
1500/1500 [==============================] - 26s - loss: 0.2976 - acc: 0.9707 - val_loss: 1.9978 - val_acc: 0.3450


In [73]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 32s - loss: 0.2194 - acc: 0.9880 - val_loss: 1.8921 - val_acc: 0.4170
Epoch 2/4
1500/1500 [==============================] - 31s - loss: 0.1784 - acc: 0.9953 - val_loss: 1.7939 - val_acc: 0.4530
Epoch 3/4
1500/1500 [==============================] - 29s - loss: 0.1376 - acc: 1.0000 - val_loss: 1.6848 - val_acc: 0.5030
Epoch 4/4
1500/1500 [==============================] - 27s - loss: 0.1064 - acc: 1.0000 - val_loss: 1.5679 - val_acc: 0.5760


In [74]:
model.optimizer.lr = 0.01
model.fit_generator(batches, batches.nb_sample, nb_epoch=8, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/8
1500/1500 [==============================] - 34s - loss: 0.0967 - acc: 1.0000 - val_loss: 1.4350 - val_acc: 0.6500
Epoch 2/8
1500/1500 [==============================] - 29s - loss: 0.0794 - acc: 1.0000 - val_loss: 1.2973 - val_acc: 0.6960
Epoch 3/8
1500/1500 [==============================] - 27s - loss: 0.0707 - acc: 1.0000 - val_loss: 1.1705 - val_acc: 0.7300
Epoch 4/8
1500/1500 [==============================] - 28s - loss: 0.0576 - acc: 1.0000 - val_loss: 1.0429 - val_acc: 0.7700
Epoch 5/8
1500/1500 [==============================] - 27s - loss: 0.0549 - acc: 1.0000 - val_loss: 0.9286 - val_acc: 0.8100
Epoch 6/8
1500/1500 [==============================] - 26s - loss: 0.0526 - acc: 1.0000 - val_loss: 0.8159 - val_acc: 0.8480
Epoch 7/8
1500/1500 [==============================] - 31s - loss: 0.0424 - acc: 1.0000 - val_loss: 0.7168 - val_acc: 0.8680
Epoch 8/8
1500/1500 [==============================] - 30s - loss: 0.0396 - acc: 1.0000 - val_loss: 0.6387 - val_acc: 0.8840


The training set here is very rapidly reaching a very high accuracy. So if we could regularize this, perhaps we could get a reasonable result.

So, what kind of regularization should we try first? As we discussed in lesson 3, we should start with data augmentation.

## Data augmentation

To find the best data augmentation parameters, we can try each type of data augmentation, one at a time. For each type, we can try four very different levels of augmentation, and see which is the best. In the steps below we've only kept the single best result we found. We're using the CNN we defined above, since we have already observed it can model the data quickly and accurately.

Width shift: move the image left and right -

In [76]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [77]:
model = conv1(batches)

Exception KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored


Epoch 1/2


KeyboardInterrupt: 

Height shift: move the image up and down -

In [65]:
gen_t = image.ImageDataGenerator(height_shift_range=0.05)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1568 images belonging to 10 classes.


In [66]:
model = conv1(batches)

Epoch 1/2
1568/1568 [==============================] - 11s - loss: 1.7843 - acc: 0.4458 - val_loss: 2.1259 - val_acc: 0.2375
Epoch 2/2
1568/1568 [==============================] - 11s - loss: 0.7028 - acc: 0.7825 - val_loss: 2.0232 - val_acc: 0.3164
Epoch 1/4
1568/1568 [==============================] - 11s - loss: 0.3586 - acc: 0.9152 - val_loss: 2.1772 - val_acc: 0.1806
Epoch 2/4
1568/1568 [==============================] - 11s - loss: 0.2335 - acc: 0.9490 - val_loss: 2.1935 - val_acc: 0.1727
Epoch 3/4
1568/1568 [==============================] - 11s - loss: 0.1626 - acc: 0.9656 - val_loss: 2.1944 - val_acc: 0.2106
Epoch 4/4
1568/1568 [==============================] - 11s - loss: 0.1214 - acc: 0.9758 - val_loss: 2.3481 - val_acc: 0.1766


Random shear angles (max in radians) -

In [67]:
gen_t = image.ImageDataGenerator(shear_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1568 images belonging to 10 classes.


In [68]:
model = conv1(batches)

Epoch 1/2
1568/1568 [==============================] - 11s - loss: 1.6148 - acc: 0.5223 - val_loss: 2.2513 - val_acc: 0.2475
Epoch 2/2
1568/1568 [==============================] - 11s - loss: 0.3915 - acc: 0.9203 - val_loss: 2.0757 - val_acc: 0.2725
Epoch 1/4
1568/1568 [==============================] - 11s - loss: 0.1478 - acc: 0.9821 - val_loss: 2.1869 - val_acc: 0.3084
Epoch 2/4
1568/1568 [==============================] - 11s - loss: 0.0831 - acc: 0.9904 - val_loss: 2.2449 - val_acc: 0.3164
Epoch 3/4
1568/1568 [==============================] - 11s - loss: 0.0530 - acc: 0.9955 - val_loss: 2.2426 - val_acc: 0.3154
Epoch 4/4
1568/1568 [==============================] - 11s - loss: 0.0343 - acc: 0.9994 - val_loss: 2.2609 - val_acc: 0.3234


Rotation: max in degrees -

In [69]:
gen_t = image.ImageDataGenerator(rotation_range=15)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1568 images belonging to 10 classes.


In [70]:
model = conv1(batches)

Epoch 1/2
1568/1568 [==============================] - 11s - loss: 1.9734 - acc: 0.3865 - val_loss: 2.1849 - val_acc: 0.3064
Epoch 2/2
1568/1568 [==============================] - 11s - loss: 0.8523 - acc: 0.7411 - val_loss: 2.0310 - val_acc: 0.2655
Epoch 1/4
1568/1568 [==============================] - 11s - loss: 0.4652 - acc: 0.8833 - val_loss: 2.0401 - val_acc: 0.2036
Epoch 2/4
1568/1568 [==============================] - 11s - loss: 0.3448 - acc: 0.9101 - val_loss: 2.2149 - val_acc: 0.1317
Epoch 3/4
1568/1568 [==============================] - 11s - loss: 0.2411 - acc: 0.9420 - val_loss: 2.2614 - val_acc: 0.1287
Epoch 4/4
1568/1568 [==============================] - 11s - loss: 0.1722 - acc: 0.9636 - val_loss: 2.1208 - val_acc: 0.2106


Channel shift: randomly changing the R,G,B colors - 

In [76]:
gen_t = image.ImageDataGenerator(channel_shift_range=20)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1568 images belonging to 10 classes.


In [77]:
model = conv1(batches)

Epoch 1/2
1568/1568 [==============================] - 11s - loss: 1.6381 - acc: 0.5191 - val_loss: 2.2146 - val_acc: 0.3483
Epoch 2/2
1568/1568 [==============================] - 11s - loss: 0.3530 - acc: 0.9305 - val_loss: 2.0966 - val_acc: 0.2665
Epoch 1/4
1568/1568 [==============================] - 11s - loss: 0.1036 - acc: 0.9923 - val_loss: 2.4195 - val_acc: 0.1766
Epoch 2/4
1568/1568 [==============================] - 11s - loss: 0.0450 - acc: 1.0000 - val_loss: 2.6192 - val_acc: 0.1667
Epoch 3/4
1568/1568 [==============================] - 11s - loss: 0.0259 - acc: 0.9994 - val_loss: 2.7227 - val_acc: 0.1816
Epoch 4/4
1568/1568 [==============================] - 11s - loss: 0.0180 - acc: 0.9994 - val_loss: 2.7049 - val_acc: 0.2206


And finally, putting it all together!

In [79]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


In [80]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 33s - loss: 2.8812 - acc: 0.1233 - val_loss: 2.4098 - val_acc: 0.0910
Epoch 2/2
1500/1500 [==============================] - 26s - loss: 2.5546 - acc: 0.1853 - val_loss: 2.2749 - val_acc: 0.1450
Epoch 1/4
1500/1500 [==============================] - 35s - loss: 2.2515 - acc: 0.2460 - val_loss: 2.2783 - val_acc: 0.1770
Epoch 2/4
1500/1500 [==============================] - 28s - loss: 2.0939 - acc: 0.2893 - val_loss: 2.2871 - val_acc: 0.1950
Epoch 3/4
1500/1500 [==============================] - 27s - loss: 2.0048 - acc: 0.3313 - val_loss: 2.2939 - val_acc: 0.2020
Epoch 4/4
1500/1500 [==============================] - 29s - loss: 1.8809 - acc: 0.3733 - val_loss: 2.2665 - val_acc: 0.2270
Epoch 1/4
1500/1500 [==============================] - 33s - loss: 1.7603 - acc: 0.3880 - val_loss: 2.2424 - val_acc: 0.2400
Epoch 2/4
1500/1500 [==============================] - 27s - loss: 1.7261 - acc: 0.4213 - val_loss: 2.2156 - val_acc: 0.2740


At first glance, this isn't looking encouraging, since the validation set is poor and getting worse. But the training set is getting better, and still has a long way to go in accuracy - so we should try annealing our learning rate and running more epochs, before we make a decisions.

In [60]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/5
1568/1568 [==============================] - 11s - loss: 1.1570 - acc: 0.6282 - val_loss: 2.4787 - val_acc: 0.1048
Epoch 2/5
1568/1568 [==============================] - 11s - loss: 1.0278 - acc: 0.6582 - val_loss: 2.4211 - val_acc: 0.1267
Epoch 3/5
1568/1568 [==============================] - 11s - loss: 0.9459 - acc: 0.6939 - val_loss: 2.5656 - val_acc: 0.1477
Epoch 4/5
1568/1568 [==============================] - 11s - loss: 0.9045 - acc: 0.6996 - val_loss: 2.2994 - val_acc: 0.2365
Epoch 5/5
1568/1568 [==============================] - 11s - loss: 0.8346 - acc: 0.7360 - val_loss: 2.1203 - val_acc: 0.2705


Lucky we tried that - we starting to make progress! Let's keep going.

In [61]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=25, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/25
1568/1568 [==============================] - 11s - loss: 0.8055 - acc: 0.7423 - val_loss: 2.0895 - val_acc: 0.2984
Epoch 2/25
1568/1568 [==============================] - 11s - loss: 0.7538 - acc: 0.7621 - val_loss: 1.8985 - val_acc: 0.4212
Epoch 3/25
1568/1568 [==============================] - 11s - loss: 0.7037 - acc: 0.7774 - val_loss: 1.7200 - val_acc: 0.4411
Epoch 4/25
1568/1568 [==============================] - 11s - loss: 0.6865 - acc: 0.7966 - val_loss: 1.5225 - val_acc: 0.5180
Epoch 5/25
1568/1568 [==============================] - 11s - loss: 0.6404 - acc: 0.8036 - val_loss: 1.3924 - val_acc: 0.5319
Epoch 6/25
1568/1568 [==============================] - 11s - loss: 0.6116 - acc: 0.8144 - val_loss: 1.4472 - val_acc: 0.5259
Epoch 7/25
1568/1568 [==============================] - 11s - loss: 0.5671 - acc: 0.8361 - val_loss: 1.4703 - val_acc: 0.5549
Epoch 8/25
1568/1568 [==============================] - 11s - loss: 0.5559 - acc: 0.8265 - val_loss: 1.2402 - val_acc:

Amazingly, using nothing but a small sample, a simple (not pre-trained) model with no dropout, and data augmentation, we're getting results that would get us into the top 50% of the competition! This looks like a great foundation for our futher experiments.

To go further, we'll need to use the whole dataset, since dropout and data volumes are very related, so we can't tweak dropout without using all the data.